In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
df = pd.read_excel(r'D:\SWORK\Тестовые задания\United Consulting Group\Transmission.xlsx', encoding='utf-8')

In [ ]:
df.info()

In [ ]:
df2 = df

In [ ]:
df2.index = df2['trackid']
df2 = df2.drop(['trackid'], axis=1)
df2.head()

In [ ]:
df3 = df2

In [ ]:
df3['rpm_dif'] = ""
df3['speed_dif'] = ""
df3['speed_dif / rpm_dif'] = ""

In [ ]:
df3.head()

In [ ]:
# df4['rpm_dif'] - Колонка в которой расчитал изменение rpm по сравнению с предыдущей точкой
df4 = df3
df4 = df4.reset_index()
df4all = df4['rpm'].iloc[:-1]
df4all[-1] = 0
df4all.index = df4all.index + 1
df4all = df4all.sort_index()
df4['rpm_dif'] = df4['rpm'] - df4all

In [ ]:
# df4['speed_dif'] - Колонка в которой расчитал изменение speed по сравнению с предыдущей точкой
df4 = df4
df4 = df4.reset_index(drop=True)
df4all = df4['speed'].iloc[:-1]
df4all[-1] = 0
df4all.index = df4all.index + 1
df4all = df4all.sort_index()
df4['speed_dif'] = df4['speed'] - df4all

In [ ]:
df4.head()

In [ ]:
"""Для классификации поездок решил сгенерировать новый признак (отношение изменение сокрости к изменению оборотов)
поскольку на вариаторе нет переключения скоростей и общая сумма по новому признаку должна быть по идее ниже.
В данных есть пропуски и при рпсчете новой фичи возникли inf / -inf. Данное значение при агрегации данных не давало
возможности нормально отобразить результаты. Для этого применил "хитрость ниже" введя в места пропуска маленькие
значения в знаменатель и обнуляя значения в числителе. (таких значений было относительно мало, посчитал подобные 
изменения допустимыми)"""

filt = (df4['rpm_dif'] ==0)&(df4['speed_dif'] != 0)
df4[filt]
df4.loc[df4[filt].index.values,'speed_dif'] = 0
df4.loc[df4[filt].index.values,'rpm_dif'] = 0.001

In [ ]:
# расчет нового признака
df4['speed_dif / rpm_dif'] = df4['speed_dif'] / df4['rpm_dif']

In [ ]:
df4.head()

In [ ]:
df4 = df4.fillna(0)

In [ ]:
df4['Marker'] = 1

In [ ]:
"""Посколку в новом признаке перемножались два столбца в которых было возможно 4 комбинации знаков:
'++', '+-', '-+', '--' то по правилам умножения у четвертой комбинации '--' получался знак плюс. Хотя для накопительного
эффекта подсчета суммы по новому признаку требовалось сохранить знак минус. Для этого ввел доп колонку в которо для
подобных случаев проставил значение -1 и в последствии на него умножил приведя значения знаков нового параметра в нужное 
состояние"""

filt = (df4['rpm_dif']<0)&(df4['speed_dif']<0)
df4[filt]
df4.loc[df4[filt].index.values,'Marker'] = -1

In [ ]:
df4['speed_dif / rpm_dif'] = df4['speed_dif / rpm_dif'] * df4['Marker']
df4['speed_dif / rpm_dif'] = df4['speed_dif / rpm_dif'].astype(float)

In [ ]:
df4.head()

In [ ]:
df5 =df4.groupby('trackid').agg({'speed_dif / rpm_dif': 'sum'})

In [ ]:
x_axis = df5.index
y_axis = df5['speed_dif / rpm_dif']
plt.scatter(x_axis, y_axis, c=df5['speed_dif / rpm_dif'])
plt.show()

In [ ]:
model = KMeans(n_clusters=2, n_init=2)
model.fit(df5)
all_predictions = model.predict(df5)
allp = pd.DataFrame(all_predictions, columns=['CLASS'])
allp = allp.assign(trackid = df5.index)
allp.index = allp['trackid']
allp = allp.drop(['trackid'], axis = 1)
df5 = pd.merge(df5, allp, left_index=True, right_index=True)
df5.head(10)

In [ ]:
x_axis = df5['speed_dif / rpm_dif']
y_axis = df5['speed_dif / rpm_dif'] 
plt.scatter(x_axis, y_axis, c=df5['CLASS'])
plt.show()

In [ ]:
df5['CLASS2'] = ''

In [ ]:
df5['CLASS2'][df5['CLASS'] == 1] = 'Механика'
df5['CLASS2'][df5['CLASS'] == 0] = 'Вариатор'

In [ ]:
df5.head(15)

In [ ]:
df5.to_excel('Харитонов_Д.В._Transmission_answer2.xlsx')